<a href="https://colab.research.google.com/github/yuen-liu/markovscribe/blob/main/Markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Methods of Text Completion: Markov Chains and LLMs
by: Tejas Tirthapura (tt3021), Bridget Gwyneth Liu (bgl2126), Richard Li (rjl2194)

# Abstract



In this study, we compare two approaches to new-word prediction in natural language. The LLM (Large Language Model) and the n-gram natural language model. First, we explore the use of Markov Chains to build an n-gram-based natural language model, which will predict the next word in a sequence based solely on the previous few words. We implement a n-gram generator, represent the learned relationships as transition matrices, and visualize them as weighted directed graphs. Our model uses a straightforward probabilistic approach grounded in the Markov property, assuming that the next state depends only on the current state, not the full history. The model learns transition probabilities over bigrams and trigrams via simple counts and smoothing, producing an interpretable probability table and weighted transition graph.
Next, we use a basic Large Language Model (LLM) using transformers for comparison. Unlike the Markov model, the LLM can theoretically capture longer dependencies and more complex patterns in text, but requires significantly more computation, larger datasets, and careful tuning to avoid overfitting.


Our findings show that Markov n-gram models are simple, fast to train, and effective on small or structured datasets, but struggle with ambiguity and long-term context. In contrast, LLM models offer more predictive power and flexibility but at the cost of higher computational requirements and longer training times. Ultimately, the Markov Chain approach provides a powerful yet interpretable baseline, while neural models offer improved performance when scaling to richer, more complex text data.

# Introduction


A Markov Chain is a mathematical system that undergoes transitions from one state to another within a finite or countable set of states (1). Markov Chains are unique because they must satisfy the Markov Property:

The future state of the system depends only on the present state, and not on the sequence of events that preceded it.

In other words, the system has no memory of the past beyond its current state. This "memorylessness" makes Markov Chains particularly useful for modeling processes where the next outcome can be predicted based purely on the present.

In the context of Natural Language Processing (NLP), a Markov Chain can be used to predict the next word in a sentence by considering only the previous word (for a bigram model) or the previous few words (for higher-order n-grams).
We represent these probabilities using a transition matrix or graph, where each node is a word or phrase, and each directed edge represents the probability of moving from one word to the next.

Thus, using Markov Chains establishes the basis of many classic language models, allowing systems to generate, autocomplete, or even correct human language input in a probabilistic, interpretable way. Today, speech recognition, handwriting recognition, information retrieval, data compression, and even spam filtering all utilize this technique (2).

In our project, we model word prediction as a Markov Chain by treating each word (or sequence of words) as a state.
The idea is that as we move through a piece of text word by word, we are essentially transitioning from one state (word) to the next.

For example, in a bigram model (n=2), the probability of seeing the next word depends only on the current word — not the entire sentence history — which exactly matches the Markov Property, or that the future state of the system depends only the present state.

Similarly, in a trigram model (n=3), the current state is the last two words, and the next word depends only on that pair, not anything before them.

Thus, we can represent our model as:

𝑃 ( next word ∣ previous word(s) ) = 𝑃 ( next state ∣ current state )

Transitions between states are determined by how often a particular word follows a given previous word (or words) in the training text.
We can store these transition probabilities in a transition matrix or transition graph, where the weights on the edges represent the likelihood of moving from one word (state) to the next.

To be more precise, we calculated words' relative frequencies to attain these transition probabilities. This is also known as a way to find the maximum likelihood estimation or MLE of a word (3). By getting the counts of each word from the sampled text, and then dividing by some total count so that the resulting probabilities fell between 0 and 1 and summed to 1, the MLE was calculated. In the next section, the precise methodology for this is described.

By learning these transitions from large text datasets, our model can then predict likely next words given a prompt — just like a classic Markov Chain moves between states based on probability.


# Markov Chain Implementation
There are 3 main parts to implementing the markov chain code for predicting the word on the previous n words.

1.   Gather the training data
2. create the transition matrix based on the training data
3. retrieve the appropriate data from the transition matrix to predict the next word

Most of this is simple enough, but a interesting challenge is determining how to store the data in an transition matrix. We decided to store this data in a dictionary, where the key is a tuple of words that represents the state, and the value is another dictionary in which the key is the next word, and the value is the count that it appears in the training data.

Lets dive into how this code is actually implemented now.

## Gathering the training data

We chose to use books from the [Gutenberg Project](https://www.gutenberg.org/) because they provide easily accessible text files of full ebooks with a large number of words. Each book is identifiable by an ID number, so we implemented a function that returns the string of a book given its specific ID. In this function, we retrieve the text content of the response, remove the header and footer, and replace all characters that are not letters, apostrophes, or spaces with a single space.

We then created another function that gathers the strings of the ten most popular books on the Gutenberg Project and returns a single combined string. These books are:
* *Frankenstein* by Mary Wollstonecraft Shelley
* *Moby Dick* by Herman Melville
* *Pride and Prejudice* by Jane Austen
* *Romeo and Juliet* by William Shakespeare
* *Alice's Adventures in Wonderland* by Lewis Carroll
* *A Doll's House* by Henrik Ibsen
* *The Great Gatsby* by F. Scott Fitzgerald
* *The Importance of Being Earnest* by Oscar Wilde
* The Complete Works of William Shakespeare
* *Middlemarch* by George Eliot



In [ ]:
import requests
import networkx as nx
import matplotlib.pyplot as plt

def get_book(book_number):
    """
    Downloads and returns the text of a Gutenberg book by its number.
    """
    response = requests.get(f"https://www.gutenberg.org/cache/epub/{book_number}/pg{book_number}.txt")
    if response.status_code != 200:
        raise Exception(f"Failed to fetch plain text file, status code {response.status_code}")

    text = response.text

    # Clean header and footer
    start_marker = " ***"
    end_marker = "*** END OF"

    start_idx = text.find(start_marker)
    end_idx = text.find(end_marker)

    if start_idx != -1 and end_idx != -1:
        book_text = text[start_idx:end_idx]
    else:
        book_text = text
    cleaned_text = ""
    for c in book_text:
      if c.isalpha() or c.isspace() or c == "'":
        cleaned_text += c
      else:
        cleaned_text += " "

    return cleaned_text.lower().strip()

def generate_all_books():
  final_string = ""
  for i in {84, 2701, 1342, 11, 1513, 64317, 2542, 844, 26184, 100}: #, 145, 2641, 174, 37106, 16389, 67979, 345, 2554, 394, 6761, 43, 6593, 2160, 4085, 5200
    final_string += get_book(i)
  return final_string

## Creating the Markov Chain

As stated before, storing the Markov chain is a complex task. Let's once again unpack the data we need to store for it to function. The Markov chain has states, which we represent as tuples of words, and each state can transition to many different words with certain probabilities. To implement this, we turn the input string into a dictionary where the keys are tuples of words and the values are themselves dictionaries that map the next word to a count of how often it appears. We use counts instead of probabilities because probabilities would require iterating through all transitions for each state and dividing by the total count—this would add extra code and reduce readability. While using a dictionary works for our purposes, it’s not the most efficient structure for lookup—finding the most frequent word requires scanning every key in the dictionary. A more optimal structure like a red-black tree could offer faster lookups, though this may be unnecessary for our current scope.

We implement the Markov chain such that it uses the previous n-1 words to predict the next word, rather than a fixed number. We define a class to represent the Markov chain, storing n in self.n and the dictionary in self.ngram_dict.

To construct this large dictionary in Python, we first split the input string into tokens. Then, for each sequence of tokens, we call the learn function. If the tuple key already exists in the dictionary, we increment the count of the following word; otherwise, we create a new entry and set its count to 1.

This leads to a slight problem when predicting the first n-1 words. If we don't have entries in the dictionary for shorter sequences, how do we generate those initial words? To solve this, we add to the dictionary not only tuples of length n-1, but also of every length from 1 to n-1. This additional data also helps when we can’t find the current state in the dictionary—we can fall back to a shorter key by trying n-2 words, then n-3, and so on, until we get results. If we still can't find a match, we resort to guessing.

Finally, we create a function called babble that takes an initial string and returns the next m words.

In [ ]:
import random

class MarkovChain:
    def __init__(self, n):
        self.n = n
        self.ngram_dict = {}  # {tuple: {next_word: count}}

    def _learn_key(self, key, value):
        if key in self.ngram_dict:
            if value in self.ngram_dict[key]:
              self.ngram_dict[key][value] += 1
            else:
              self.ngram_dict[key][value] = 1
        else:
            self.ngram_dict[key] = {value: 1}

    def learn(self, text):
        words = text.split()
        # Create overlapping n-grams
        for k in range(1, self.n + 1):
          for i in range(len(words) - k + 1):
              ngram = words[i:i + k]
              key = tuple(ngram[:-1])  # n-1 words as key
              value = ngram[-1]         # nth word as value
              self._learn_key(key, value)

    def _next_word(self, current_state):
        # Convert current_state (string) to a tuple of words
        state_words = current_state.split()
        key = tuple(state_words[-(self.n - 1):])
        len_offset = 1;
        while (key not in self.ngram_dict):
          key = tuple(state_words[-(self.n - 2 - len_offset):])
          len_offset += 1
          if (len_offset > self.n - 2):
            # Fallback: pick a random key's most common word
            random_key = random.choice(list(self.ngram_dict.keys()))
            return self._next_word(" ".join(random_key))  # Recurse with valid key

        nw = ""
        nw_frequency = -1

        for k in self.ngram_dict[key]:
            if self.ngram_dict[key][k] > nw_frequency:
              nw = k
              nw_frequency = self.ngram_dict[key][k]
        return nw

    def babble(self, amount, state=""):
        if amount <= 0:
            return state.strip()
        next_word = self._next_word(state)
        if not next_word:
            return state.strip()
        new_state = f"{state} {next_word}".strip()
        return self.babble(amount - 1, new_state)


In [ ]:
trainingdata = generate_all_books()

In [ ]:
mc2 = MarkovChain(2)
mc2.learn(trainingdata)
mc3 = MarkovChain(3)
mc3.learn(trainingdata)
mc4 = MarkovChain(4)
mc4.learn(trainingdata)
mc6 = MarkovChain(6)
mc6.learn(trainingdata)
mc8 = MarkovChain(8)
mc8.learn(trainingdata)

In [ ]:

print(f"Mc2: {mc2.babble(20, 'i')}")
print(f"Mc3: {mc3.babble(20, 'i')}")
print(f"Mc4: {mc4.babble(20, 'i')}")
print(f"Mc6: {mc6.babble(20, 'i')}")
print(f"Mc8: {mc8.babble(20, 'i')}")

Mc2: i am i am i am i am i am i am i am i am i am i am i
Mc3: i am not in the world s a good deal of good hope and fear not my lord i ll be
Mc4: i am not in the least of all these piteous woes we cannot without circumstance descry re enter some of the
Mc6: i am not in the mind but i were better to be married of him than of another for he is
Mc8: i am not in the mind but i were better to be married of him than of another for he is


## Key Observations:
In the bigram (Mc2) model the text tends to be repetitive with very little context. This is because in a bigram model the next word is only based on the previous word in the sentance and the most likely word to follow I is am and vice versa, creating an infinite loop.

In a mid-sized n-gram model like Mc3 or Mc4, there is less repetition and more variety. This is because there is adequate data to create the markov chain where the model can provide more than one result for the given state and not get stuck in a loop.

For longer n-grams (Mc6, Mc8), as the n-gram size increases, the output becomes more contextually relevant, with longer phrases and more coherent text. However, for this specific input data, the models spit out the exact same text from Shakespeare's "As You Like It". This is becuase the input data is not big enough for the model to have more than one result given such a long string of words before it. The longer n-gram models provide more coherent text but still rely heavily on common phrases found in the training data. The improvements in text quality are clear but limited by the data.

# LLM Implementation

A Large Language Model (LLM) is a type of deep learning model trained on vast amounts of text data to understand, generate, and manipulate human language. Built on the transformer architecture, LLMs capture complex relationships between words in a given context. During pretraining, the model learns to predict the next word or token in a sequence, which enables it to generate coherent and contextually appropriate text. Our implementation of an LLM involves loading a pretrained model and tokenizer from Hugging Face library. The input text is first tokenized into numerical representations, then passed into the model, which generates output tokens based on learned patterns. These tokens are decoded back into text, producing results such as completions like what the markov chain does.

In [ ]:
!pip install transformers torch


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

class LLMBabbler:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
        self.model = AutoModelForCausalLM.from_pretrained("distilgpt2")
        self.model.eval()
        self.model.to("cuda" if torch.cuda.is_available() else "cpu")

    def _get_next_word(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=1,
                do_sample=False
            )
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return decoded[len(prompt):].strip()

    def babble(self, start_text, num_words):
        text = start_text.strip()
        for _ in range(num_words):
            next_word = self._get_next_word(text)
            if not next_word:
                break
            text += " " + next_word
        return text


In [ ]:
llm = LLMBabbler()
print(llm.babble("i am", 20))



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

i am i am a bit of a fan of the game , but I 'm not sure if I 'm a




# Comparison between Markov Chain and LLM

The LLM clearly has more creativity and variety in its response, but when starting with such a small start text, it is limited in what it can come up with. A LLM likely would serve much better for longer start texts and tasks like answering questions, while the markov chain serves best for quick completions based on a short context. The LLM also takes a much longer time to train. It doesn't take long in our code because we use a pretrained LLM, while we gather our own data and train our markov chain with it.


# Further Implications

Building an n-gram language model using Markov Chains opens up several broader possibilities and connections.

Understanding Language Generation: Simple Markov models show how coherent-seeming text can be generated purely based on local word patterns without needing deep understanding, highlighting the structure embedded even in casual human writing. For example, bigrams and trigrams can be reasonably accurate in reproducing plausible word sequences (in the, out of, etc). This extends to larger word sequences. Even “memoryless” models can retain the human-aspect of text through the probabilities of specific words following other words.

Baseline for More Complex Models: Markov-based models serve as a strong, interpretable baseline for language modeling, against which more complex models like RNNs and Transformers can be evaluated. N-grams models are beneficial because they are so simple. This means that, in the future, more complex models can be compared to the simple “vanilla” model. If a more “complex” model underperforms compared to the basic n-gram model, then it’s an indicator that the architecture has either a fundamental problem, or is too complex for its purpose.

Applications to Autocomplete and Spell Checking: Predicting the next word based on previous context is the foundation for many real-world applications, such as phone keyboards, search engine suggestions, and smart email composition. More importantly, n-gram lookups are incredibly efficient, which is beneficial for devices with tight latency and limited memory. This can be important for potentially doing word prediction, like in iphone keyboard functions, or in search-engine autocomplete.

Insights into Data Sparsity and Smoothing: Higher-order n-gram models have challenges with data sparsity; 3-word and 4-word probability sequences are unlikely and almost impossible respectively. When uncorrected, random backoffs are inevitable, as the model tries to assign probabilities to these events. Future developments into solutions like smoothing, backoff models, and more general machine learning techniques will happen to create methods that deal with these rare events.

Connections to Modern Deep Learning: Although simple, n-gram models using Markov Chains reflect early versions of what modern deep learning models attempt to generalize: learning transition patterns between tokens in a sequence (which is now done in much more powerful, distributed ways). The core of the algorithm still exists. RNNs and Transformers, by allowing for recurrence or self-attention, side-step the downsides of n-grams and are a future connection.

Extensions to Other Fields: Markov Chains aren't limited to text. This type of modeling appears in genetics (predicting DNA sequences), economics (predicting market states), game AI (predicting player moves), and more.



# Conclusion


Though simple, our Markov Chain-based n-gram model offers key insights into how language can be statistically modeled and predicted. By working through this project, we not only explore the foundational ideas behind modern language processing systems but also build a practical, interpretable system that highlights both the power and the limitations of memoryless models. This hands-on understanding prepares us to appreciate and engage with more advanced methods, such as Recurrent Neural Networks and Transformer-based architectures, that build on and extend these ideas.

# References



1. https://www.publichealth.columbia.edu/research/population-health-methods/markov-chain-monte-carlo
2. https://www.mecs-press.org/ijitcs/ijitcs-v14-n2/IJITCS-V14-N2-1.pdf
3. https://web.stanford.edu/~jurafsky/slp3/3.pdf
4. https://arxiv.org/html/2410.02724v1
5. https://web.stanford.edu/~jurafsky/slp3/3.pdf
6. https://sookocheff.com/post/nlp/
7. https://www.gutenberg.org/ebooks/search/?sort_order=downloads

